# Google Trends Local Deals Restuarants and Food Scripts 


In [1]:
from datetime import date
import pandas as pd
import pytrends
from pytrends.request import TrendReq
import datetime 

In [2]:
numyears = 1
numdays = 7
numweeks = 52
total_time_range = numdays * numweeks * numyears

end_date= date.today()
today_date = date.today()
end_date = today_date
begin_date = end_date - datetime.timedelta(days = total_time_range-7)

user_timeframe = begin_date.strftime('%Y-%m-%d')+' '+end_date.strftime('%Y-%m-%d')
print(user_timeframe)

2019-03-25 2020-03-16


In [3]:
# initiate API 
pytrend =  TrendReq()

In [4]:
# perform initial search for related queries 
pytrend.build_payload(kw_list=['food', 'best', 'restaurants', 'cafe', 'vegan'], geo = 'GB', timeframe = user_timeframe, cat = 71)
related_queries_dict = pytrend.related_queries()
related_topics_dict = pytrend.related_topics()

In [5]:
related_queries_dict

{'food': {'top':                      query  value
  0             food near me    100
  1                 bbc food     87
  2                good food     81
  3            food delivery     58
  4            bbc good food     54
  5             chinese food     36
  6             food recipes     36
  7   marks and spencer food     32
  8                fast food     30
  9                 m&s food     28
  10              vegan food     26
  11             indian food     25
  12              party food     23
  13        food restaurants     20
  14              order food     18
  15              tesco food     18
  16             street food     18
  17               thai food     17
  18           takeaway food     16
  19               free food     15
  20              food ideas     15
  21               asda food     14
  22   food delivery near me     14
  23         bbc food recipe     13
  24             food places     12,
  'rising':                               query 

In [6]:

def get_data(dicti, key):
    
    df1 = related_queries_dict.get(key).get('rising')
    df2 = related_queries_dict.get(key).get('top')
    
    df1 = pd.DataFrame(df1, columns=['query', 'value'])
    df2 = pd.DataFrame(df2, columns=['query', 'value'])
    
    
    return df1, df2

In [7]:
best_rising, best_top = get_data(related_queries_dict, key='best')
food_rising, food_top = get_data(related_queries_dict, key='food')
rest_rising, rest_top = get_data(related_queries_dict, key='restaurants')
cafe_rising, cafe_top = get_data(related_queries_dict, key='cafe')
vega_rising, vega_top = get_data(related_queries_dict, key='vegan')


In [8]:
def prep_data(df, key):
    
    searches = df[key].to_list()
    
    groupkeywords = list(zip(*[iter(searches)]*1))
    groupkeywords = [list(x) for x in groupkeywords]
    
    return groupkeywords, searches

In [9]:
best_risng_kws, searches_best = prep_data(best_rising, key='query')
food_risng_kws, searches_food = prep_data(food_rising, key='query')
rest_risng_kws, searches_rest = prep_data(rest_rising, key='query')
cafe_risng_kws, searches_cafe = prep_data(cafe_rising, key='query')
vega_risng_kws, searches_vega = prep_data(vega_rising, key='query')

In [10]:
def perform_trending_search(groupkeywords, searches, filename):
    
    dicti = {}
    
    i = 1
    
    for trending in groupkeywords:
        pytrend.build_payload(trending, timeframe = user_timeframe, geo = 'GB', cat = 71)
        dicti[i] = pytrend.interest_over_time()
        i+=1
    
    result = pd.concat(dicti, axis=1)
    result.columns = result.columns.droplevel(0)
    result = result.drop('isPartial', axis = 1)
    result.reset_index(level=0, inplace=True)
    result = pd.melt(result, id_vars='date', value_vars=searches)
    
    save_file = result.to_csv(filename)
    
    return result



In [11]:
# have to split and run in different cells or google will block the request
perform_trending_search(groupkeywords=best_risng_kws, searches=searches_best, filename='RisingLocalDealsBest.csv')
perform_trending_search(groupkeywords=rest_risng_kws, searches=searches_rest, filename='RisingLocalDealsRest.csv')
perform_trending_search(groupkeywords=food_risng_kws, searches=searches_food, filename='RisingLocalDealsFood.csv')

,date,variable,value
0,2019-03-31,beer munch food,0
1,2019-04-07,beer munch food,0
2,2019-04-14,beer munch food,0
3,2019-04-21,beer munch food,0
4,2019-04-28,beer munch food,0
...,...,...,...
1245,2020-02-09,m&s food near me,35
1246,2020-02-16,m&s food near me,21
1247,2020-02-23,m&s food near me,24
1248,2020-03-01,m&s food near me,28


In [12]:
perform_trending_search(groupkeywords=cafe_risng_kws, searches=searches_cafe, filename='RisingLocalDealsCafe.csv')
perform_trending_search(groupkeywords=vega_risng_kws, searches=searches_vega, filename='RisingLocalDealsVega.csv')

,date,variable,value
0,2019-03-31,greggs vegan steak bake,0
1,2019-04-07,greggs vegan steak bake,0
2,2019-04-14,greggs vegan steak bake,2
3,2019-04-21,greggs vegan steak bake,0
4,2019-04-28,greggs vegan steak bake,1
...,...,...,...
1245,2020-02-09,vegan tuna,23
1246,2020-02-16,vegan tuna,19
1247,2020-02-23,vegan tuna,16
1248,2020-03-01,vegan tuna,20
